<a href="https://colab.research.google.com/github/Coffinbrain/lessons/blob/main/imdb_rating_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset:https://www.kaggle.com/datasets/darshan1504/imdb-movie-reviews-2021/code
simpletransformers:https://github.com/ThilinaRajapakse/simpletransformers

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 

In [2]:
import os
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import classification_report

In [3]:
# Enable more detailed error messages for CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
# Connect to the SQLite database
conn = sqlite3.connect('/content/drive/MyDrive/IMDB_Movies_2021.db')

In [5]:
# Load the data into a DataFrame
df = pd.read_sql_query("SELECT AUTHOR, TITLE, REVIEW, RATING FROM REVIEWS", conn)
conn.close()

In [6]:
# Data preprocessing
df = df.dropna(subset=['RATING'])
df['RATING'] = df['RATING'].astype(float).round().astype(int)
df['REVIEW'] = df['REVIEW'].astype(str)

<ipython-input-6-bd39b555f581>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RATING'] = df['RATING'].astype(float).round().astype(int)
<ipython-input-6-bd39b555f581>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['REVIEW'] = df['REVIEW'].astype(str)


In [7]:
# Check and adjust label range if necessary
df['RATING'] = df['RATING'] - df['RATING'].min()  # Adjust labels to start from 0

In [8]:
# Verify the range of labels
print(f"Unique labels: {df['RATING'].unique()}")

Unique labels: [4 7 3 5 8 6 2 0 1 9]


In [9]:
# Split the data
train_df, test_df = train_test_split(df, test_size=0.3)

In [10]:
# Model configuration
model_args = {
    "num_train_epochs": 3,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "no_save": True,
}


In [11]:
# Initialize the model
model = ClassificationModel("roberta", "roberta-base", num_labels=df['RATING'].nunique(), args=model_args, use_cuda=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
# Train the model
model.train_model(train_df[['REVIEW', 'RATING']])

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/467 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/467 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/467 [00:00<?, ?it/s]

(1401, 1.6319920977552307)

In [13]:
# Predictions
predictions, raw_outputs = model.predict(test_df['REVIEW'].tolist())

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
# Compute metrics
print(classification_report(test_df['RATING'], predictions, digits=4))

              precision    recall  f1-score   support

           0     0.5432    0.8012    0.6474       322
           1     0.2295    0.1707    0.1958       164
           2     0.1500    0.0643    0.0900       140
           3     0.3415    0.1273    0.1854       110
           4     0.2123    0.3039    0.2500       102
           5     0.3333    0.1488    0.2057       121
           6     0.3333    0.4681    0.3894       141
           7     0.2961    0.2896    0.2928       183
           8     0.4062    0.0942    0.1529       138
           9     0.4642    0.7598    0.5763       179

    accuracy                         0.3912      1600
   macro avg     0.3310    0.3228    0.2986      1600
weighted avg     0.3584    0.3912    0.3479      1600

